<a href="https://colab.research.google.com/github/OgbondaGlory/3D-3D_Image_Registration/blob/main/Copy_of_Deformable_Image_Registration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 89.6 gigabytes of available RAM

You are using a high-RAM runtime!


In [5]:
!pip install pydicom

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 32.5 MB/s eta 0:00:00


In [ ]:
import os
import numpy as np
import pydicom
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import ipyvolume as ipv
import SimpleITK as sitk
import ipywidgets as widgets

# Loading the Data
data_path = "/content/drive/MyDrive/3D-3D_Image_Registration/3D-3D_Image_Registration/SE000003"

# List all files in the directory
files = os.listdir(data_path)
print(files)

def segment_lung(image):
    # Apply a threshold to separate lung pixels from others
    thresh_filter = sitk.ThresholdImageFilter()
    thresh_filter.SetLower(-1000)
    thresh_filter.SetUpper(-400)
    thresh_img = thresh_filter.Execute(image)

    # Apply morphological opening
    morph_filter = sitk.BinaryMorphologicalOpeningImageFilter()
    morph_img = morph_filter.Execute(thresh_img)

    # Apply Connected Component Labeling
    cc_filter = sitk.ConnectedComponentImageFilter()
    cc_img = cc_filter.Execute(morph_img)

    # Get the two largest components (assumed to be the lungs)
    relabel_filter = sitk.RelabelComponentImageFilter()
    relabel_img = relabel_filter.Execute(cc_img)

    # Get labels array
    labels_array = sitk.GetArrayFromImage(relabel_img)

    # Count the number of pixels in each region and sort by size
    unique, counts = np.unique(labels_array, return_counts=True)
    sorted_counts = sorted(zip(counts, unique), reverse=True)

    # Keep the two largest components (excluding background)
    lung_mask = np.isin(labels_array, [label for _, label in sorted_counts[1:3]])

    # Convert to SimpleITK image for further processing
    lung_mask_sitk = sitk.GetImageFromArray(lung_mask.astype(np.uint8))
    lung_mask_sitk.CopyInformation(image)

    return lung_mask_sitk

def segment_bones(image):
    # Convert the SimpleITK image to a numpy array
    image_array = sitk.GetArrayFromImage(image)

    # Use thresholding to identify the bones
    # Bones have higher HU than soft tissues or air
    # This range might need adjustment depending on the specific scan
    bone_threshold = 300
    bone_mask = image_array > bone_threshold

    # Convert to SimpleITK image for further processing
    bone_mask_sitk = sitk.GetImageFromArray(bone_mask.astype(np.uint8))
    bone_mask_sitk.CopyInformation(image)

    return bone_mask_sitk


def load_dicom_series(directory_path):
    reader = sitk.ImageSeriesReader()
    dicom_names = reader.GetGDCMSeriesFileNames(directory_path)
    reader.SetFileNames(dicom_names)
    image = reader.Execute()
    return image

def display_images(image, title, montage_slices=10):
    # Convert the SimpleITK image to a numpy array
    image_array = sitk.GetArrayFromImage(image)

    # Normalize to 0-255
    image_array = ((image_array - image_array.min()) * (1/(image_array.max() - image_array.min()) * 255)).astype('uint8')

    # 3D visualization using ipyvolume
    ipv.figure()
    ipv.volshow(image_array, level=[0.25, 0.75], opacity=0.03, level_width=0.1, data_min=0, data_max=255)
    fig3d = ipv.gcc()

    # Create a layout with a title
    title_widget = widgets.HTML(f'<h2>{title} - 3D View</h2>')
    layout = widgets.VBox([title_widget, fig3d])
    display(layout)

    # Montage of slices using matplotlib
    fig = plt.figure(figsize=(10, 2))
    fig.suptitle(title + " - Slice View")
    slice_interval = image_array.shape[0] // montage_slices
    for i in range(montage_slices):
        ax = fig.add_subplot(1, montage_slices, i + 1)
        ax.imshow(image_array[i * slice_interval], cmap='gray')
        plt.axis('off')
    plt.show()


# Load the DICOM images
ct_image = load_dicom_series(data_path)

# Perform lung segmentation
lung_mask = segment_lung(ct_image)

# Convert the SimpleITK image to a numpy array for visualization
lung_mask_array = sitk.GetArrayFromImage(lung_mask)

# Display segmented lungs in 3D
display_images(lung_mask, "Segmented Lungs")



# Perform bone segmentation
bone_mask = segment_bones(ct_image)

## Display the bone segmentation in 3D and slices
display_images(bone_mask, "Segmented Bones")

# Paths to the DICOM directories
fixed_image_path = "/content/drive/MyDrive/3D-3D_Image_Registration/3D-3D_Image_Registration/SE000003"
moving_image_path = "/content/drive/MyDrive/3D-3D_Image_Registration/3D-3D_Image_Registration/SE000003"

# Load the DICOM images
fixed_image = load_dicom_series(fixed_image_path)
moving_image = load_dicom_series(moving_image_path)

# Display the images with appropriate titles
display_images(fixed_image, "Fixed Image patient's CT scan")
display_images(moving_image, "Moving Image CT scan of the phantom")

# Apply the transformation
registration_method = sitk.ImageRegistrationMethod()
registration_method.SetMetricAsMeanSquares()
registration_method.SetOptimizerAsGradientDescent(learningRate=1.0, numberOfIterations=100, convergenceMinimumValue=1e-6, convergenceWindowSize=10)
registration_method.SetInterpolator(sitk.sitkLinear)

initial_transform = sitk.CenteredTransformInitializer(fixed_image,
                                                      moving_image,
                                                      sitk.Euler3DTransform(),
                                                      sitk.CenteredTransformInitializerFilter.GEOMETRY)

registration_method.SetInitialTransform(initial_transform, inPlace=False)
final_transform_v1 = registration_method.Execute(sitk.Cast(fixed_image, sitk.sitkFloat32),
                                                 sitk.Cast(moving_image, sitk.sitkFloat32))

# Resample the moving image onto the fixed image's grid
resampler = sitk.ResampleImageFilter()
resampler.SetReferenceImage(fixed_image)
resampler.SetTransform(final_transform_v1)
resampled_moving_image = resampler.Execute(moving_image)

# Display the images after transformation
display_images(fixed_image, "Fixed Image after Transformation")
display_images(resampled_moving_image, "Resampled Moving Image after Transformation")


['CT000353', 'CT000267', 'CT000195', 'CT000214', 'CT000094', 'CT000058', 'CT000021', 'CT000229', 'CT000205', 'CT000196', 'CT000347', 'CT000206', 'CT000341', 'CT000334', 'CT000111', 'CT000344', 'CT000317', 'CT000163', 'CT000337', 'CT000339', 'CT000350', 'CT000256', 'CT000177', 'CT000015', 'CT000210', 'CT000262', 'CT000008', 'CT000293', 'CT000201', 'CT000271', 'CT000323', 'CT000007', 'CT000274', 'CT000150', 'CT000328', 'CT000187', 'CT000127', 'CT000368', 'CT000166', 'CT000285', 'CT000033', 'CT000270', 'CT000189', 'CT000084', 'CT000194', 'CT000114', 'CT000057', 'CT000246', 'CT000112', 'CT000055', 'CT000100', 'CT000167', 'CT000359', 'CT000222', 'CT000379', 'CT000157', 'CT000237', 'CT000265', 'CT000226', 'CT000362', 'CT000118', 'CT000296', 'CT000386', 'CT000051', 'CT000234', 'CT000018', 'CT000298', 'CT000380', 'CT000221', 'CT000292', 'CT000067', 'CT000314', 'CT000272', 'CT000273', 'CT000160', 'CT000048', 'CT000331', 'CT000355', 'CT000028', 'CT000240', 'CT000014', 'CT000185', 'CT000178', 'CT

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


______________________

## Segmentation of the lungs